# PHSCS 530
## HW 4: Linear Algebra
#### Logan Mathews

import numpy as np.

### Problem 1: Condition number, accuracy, and interative updating

In [3]:
A = np.array([[0.780, 0.563],
             [0.913, 0.659]]) # matrix A
b = np.array([0.217,0.254]).reshape((2, 1)) # vector b

x = np.array([1, -1]).reshape((2, 1)) # exact solution of Ax = b
x_alpha = np.array([0.999, -1.001]).reshape((2, 1)) # approximate solution
x_beta = np.array([0.341, -0.087]).reshape((2, 1)) # another approximate solution

#### Part A: Residuals

In [6]:
r_alpha = b - A@x_alpha
r_beta = b - A@x_beta

print('The residual for x_alpha is',r_alpha)
print('and the residual for r_beta is',r_beta)

The residual for x_alpha is  [[0.001343]
 [0.001572]]
and the residual for r_beta is  [[1.e-06]
 [0.e+00]]


Curiously, the residual for the less-exact approximate solution ($\beta$) is smaller.

#### Part B: Condition Number

In [29]:
U, S, Vh = np.linalg.svd(A) # do SVD on matrix A
kappa = S[0]/S[1]
print('The condition number of A is',kappa)
print('We can expect to lose around k =', np.log10(kappa),' digits of accuracy.')

The condition number of A is 2193218.9997129813
We can expect to lose around k = 6.341081999492477  digits of accuracy.


Yikes! That's a wild condition number! $A$ is definitely ill-conditioned. Using the rule of thumb, we can expect to loose up to 6 digits of accuracy in our calculations due to roundoff errors.

#### Part C: Black-Box Solver

In [27]:
x_solve = np.linalg.solve(A,b)
err = x - x_solve
print('The error between the exact and solved solution is',err)

The error between the exact and solved solution is [[ 2.30988562e-11]
 [-3.20018456e-11]]


In [28]:
soln = A@x_solve - b
print('Calculating Ax - b yields',soln)

Calculating Ax - b yields [[-2.77555756e-17]
 [ 0.00000000e+00]]


It appears that the solution is plenty accurate, as we get within machine precision of zero from $Ax-b$, so the problem seems to be coming from rounding errors in calculating $A$ and $b$. This jives with what we found from the condition number, since we're getting errors in the range of $10^{-11}$ and machine precision is manifesting here as $~10^{-17}$. We lost six digits of precision, which is what we predicted in Part B from the condition number.

#### Part D: Iterative Improvement

In [37]:
Ainv = np.linalg.inv(A);
r_beta = b - A@x_beta
deltax = Ainv@(-r_beta)
xnew = x_beta - deltax
print(xnew)

[[ 1.]
 [-1.]]


Analytically, $r_\beta$ is 
$$r_\beta=b-Ax_\beta\\=\begin{bmatrix}0.217\\0.254\end{bmatrix}-\begin{bmatrix}0.780&0.563\\0.913&0.659\end{bmatrix}\begin{bmatrix}0.341\\-0.087\end{bmatrix}\\=\begin{bmatrix}0.217\\0.254\end{bmatrix}-\begin{bmatrix}0.216999\\0.254\end{bmatrix}\\=\begin{bmatrix}0.000001\\0\end{bmatrix}.$$

In [38]:
r_beta_analy = np.array([0.000001,0]).reshape((2, 1)) # vector b
deltax = Ainv@(-r_beta)
xnew = x_beta - deltax
print(xnew)

[[ 1.]
 [-1.]]


### Problem 2: Sherman-Morrison Formula

In [3]:
E = 3
t = 1

T = np.array([[E-t, t, 0, 0, 0],
              [t, E-t, t, 0, 0],
              [0, t, E-t, t, 0],
              [0, 0, t, E-t, t],
              [0, 0, 0, t, E-t]])

C = np.array([[E-t, t, 0, 0, t],
              [t, E-t, t, 0, 0],
              [0, t, E-t, t, 0],
              [0, 0, t, E-t, t],
              [t, 0, 0, t, E-t]])

Tinv = np.linalg.inv(T) # calculate the inverse of T using numpy linalg inverse routine

We're going to have to apply Sherman-Morrison twice, since we can't get directly from $T$ to $C$ by adding the outer product of two vectors. So, we start with
$D = T + u v^T$ where $u = [t,0,0,0,0]^T$ and $v = [0,0,0,0,t]^T$, hence
$$D = T + u v^T \\
= \begin{bmatrix}E-t& t& 0& 0& 0\\
              t& E-t& t& 0& 0\\
              0& t& E-t& t& 0\\
              0& 0& t& E-t& t\\
              0& 0& 0& t& E-t\end{bmatrix} + \begin{bmatrix}t\\0\\0\\0\\0\end{bmatrix}\begin{bmatrix}0&0&0&0&t\end{bmatrix}\\
              = \begin{bmatrix}E-t& t& 0& 0& 0\\
              t& E-t& t& 0& 0\\
              0& t& E-t& t& 0\\
              0& 0& t& E-t& t\\
              0& 0& 0& t& E-t\end{bmatrix} + \begin{bmatrix}0& 0& 0& 0& t\\
              0& 0& 0& 0& 0\\
              0& 0& 0& 0& 0\\
              0& 0& 0& 0& 0\\
              0& 0& 0& 0& 0\end{bmatrix} \\= \begin{bmatrix}E-t& t& 0& 0& t\\
              t& E-t& t& 0& 0\\
              0& t& E-t& t& 0\\
              0& 0& t& E-t& t\\
              0& 0& 0& t& E-t\end{bmatrix}$$
              
Then, we can get to C by having
$$C = D + v u^T \\= \begin{bmatrix}E-t& t& 0& 0& t\\
              t& E-t& t& 0& 0\\
              0& t& E-t& t& 0\\
              0& 0& t& E-t& t\\
              0& 0& 0& t& E-t\end{bmatrix} + \begin{bmatrix}0\\0\\0\\0\\t\end{bmatrix}\begin{bmatrix}t&0&0&0&0\end{bmatrix}\\
              = \begin{bmatrix}E-t& t& 0& 0& t\\
              t& E-t& t& 0& 0\\
              0& t& E-t& t& 0\\
              0& 0& t& E-t& t\\
              0& 0& 0& t& E-t\end{bmatrix} + \begin{bmatrix}0& 0& 0& 0& 0\\
              0& 0& 0& 0& 0\\
              0& 0& 0& 0& 0\\
              0& 0& 0& 0& 0\\
              t& 0& 0& 0& 0\end{bmatrix} \\= \begin{bmatrix}E-t& t& 0& 0& t\\
              t& E-t& t& 0& 0\\
              0& t& E-t& t& 0\\
              0& 0& t& E-t& t\\
              t& 0& 0& t& E-t\end{bmatrix}$$

In [7]:
u = np.array([t,0,0,0,0]).reshape((5, 1)) # vector u
v = np.array([0,0,0,0,t]).reshape((5, 1)) # vector v

Dinv = Tinv - (Tinv@u@np.transpose(v)@Tinv)/(1 + np.transpose(v)@Tinv@u) # intermediate calculation of Dinv
Cinv = Dinv - (Dinv@v@np.transpose(u)@Dinv)/(1 + np.transpose(u)@Dinv@v) # inverse of C

Delta_SM = Tinv - Cinv
Cinv_direct = np.linalg.inv(C)
Delta_direct = Tinv - Cinv_direct
print(Delta_direct - Delta_SM)

[[ 0.00000000e+00  0.00000000e+00  5.55111512e-17  0.00000000e+00
  -4.44089210e-16]
 [ 1.11022302e-16 -2.22044605e-16  0.00000000e+00 -1.11022302e-16
   1.11022302e-16]
 [-1.94289029e-16  2.22044605e-16  0.00000000e+00  0.00000000e+00
   2.22044605e-16]
 [ 2.22044605e-16 -1.11022302e-16 -1.11022302e-16  0.00000000e+00
  -3.33066907e-16]
 [-2.22044605e-16  2.22044605e-16 -5.55111512e-17  0.00000000e+00
   4.44089210e-16]]


So, $\Delta_\textrm{direct}-\Delta_\textrm{S-M}=\left(T^{-1}-C^{-1}_\textrm{direct}\right)-\left(T^{-1}-C^{-1}_\textrm{S-M}\right)\approx[0]$, that is, the difference between the direct inversion of $C$ and that obtained through Sherman-Morrison are within machine precision of each other, so they are effectively the same. The only difference, then, is that we didn't have to invert $C$ for the Sherman-Morrison method.